In [2]:
# imports
import requests
import pandas as pd
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
# Load bike station data from exported .csv (See citybikes.ipynb)
station_data = pd.read_csv('../data/citybike_stations.csv')

# Format the latitude & longitude of each station for use in our API calls
station_coordinates = []
for index, row in station_data.iterrows():
    # Gather the lat and long for each station
    lat = row['latitude']
    lon = row['longitude']
    # Format the values into a single string
    formatted_coords = f"{lat},{lon}"
    # Add it to our coordinate list
    station_coordinates.append(formatted_coords)

# Headers & parameters needed for FourSquare
headers = {"Accept": "application/json"}
headers['Authorization'] = os.environ["FOURSQUARE_API_KEY"]
radius_from_location = 1000 # Distance of POI from bike station location (in meters)

# Send an API request for every pair of station coordinates to compile POI data
poi_data = []
for station_coord in station_coordinates:
    # Create the API call using the bike station location and provided search radius
    request_url = "https://api.foursquare.com/v3/places/search?ll=" + station_coord + "&radius=" + str(radius_from_location)
    response = requests.get(request_url, headers=headers)
    poi_data.append(response.json())

Load the API data into a DataFrame

In [23]:
# NOTE:seperate code block so I don't have to keep calling API
# There may be a better way to do the below, but I had a lot of issues getting this to work so I am leaving as-is :D

# Load the compiled data into Pandas dataframe 
poi_data_df = pd.DataFrame(poi_data)

# Use the sum function to flatten the results column (ie stop it from being a list of lists)
poi_data_flattened = sum(poi_data_df['results'], [])

# Load the organized data back into a dataframe
poi_data_df = pd.DataFrame(poi_data_flattened)

# Display results
poi_data_df

,fsq_id,categories,chains,distance,geocodes,link,location,name,related_places,timezone
0,54c7cade498e300defb0985f,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",[],31,"{'main': {'latitude': 55.858485, 'longitude': ...",/v3/places/54c7cade498e300defb0985f,"{'address': '6 Wilson St', 'admin_region': 'Sc...",Wilson Street Pantry,{},Europe/London
1,4bc35be22a89ef3b08e7f488,"[{'id': 13199, 'name': 'Indian Restaurant', 'i...",[],34,"{'main': {'latitude': 55.858078, 'longitude': ...",/v3/places/4bc35be22a89ef3b08e7f488,"{'address': '44 Candleriggs', 'admin_region': ...",The Dhabba,{},Europe/London
2,4b9cecbcf964a520bd8236e3,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],43,"{'main': {'latitude': 55.858429, 'longitude': ...",/v3/places/4b9cecbcf964a520bd8236e3,"{'address': '71-73 Albion St', 'admin_region':...",Merchant Square,{'children': [{'fsq_id': '595f841bf79faa623f93...,Europe/London
3,4ba0e8c3f964a520508637e3,"[{'id': 10037, 'name': 'Concert Hall', 'icon':...",[],98,"{'main': {'latitude': 55.85886, 'longitude': -...",/v3/places/4ba0e8c3f964a520508637e3,"{'admin_region': 'Scotland', 'country': 'GB', ...",City Halls,{},Europe/London
4,554cdd3c498ee17d5a870117,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...",[],144,"{'main': {'latitude': 55.859412, 'longitude': ...",/v3/places/554cdd3c498ee17d5a870117,"{'address': '127 Candleriggs', 'admin_region':...",Spitfire Espresso,{},Europe/London
...,...,...,...,...,...,...,...,...,...,...
1051,06b2e83c632e418806153b09,"[{'id': 17010, 'name': 'Used Car Dealership', ...",[],421,"{'main': {'latitude': 55.845601, 'longitude': ...",/v3/places/06b2e83c632e418806153b09,"{'address': '136 Boden St', 'admin_region': 'S...",London Cab Sales,{},Europe/London
1052,c6e7c00cef054a0a92e80063,"[{'id': 16037, 'name': 'Playground', 'icon': {...",[],543,"{'main': {'latitude': 55.842511, 'longitude': ...",/v3/places/c6e7c00cef054a0a92e80063,"{'address': '421 Baltic St', 'admin_region': '...",Baltic Street Adventure Playground,{},Europe/London
1053,4c1ae6aeb9f876b044847946,"[{'id': 12072, 'name': 'Police Station', 'icon...",[],557,"{'main': {'latitude': 55.848546, 'longitude': ...",/v3/places/4c1ae6aeb9f876b044847946,"{'address': '851 London Rd', 'admin_region': '...",London Road Police Station,{},Europe/London
1054,5bc4936d898bdc002bfa5d46,"[{'id': 15003, 'name': 'Alternative Medicine C...",[],666,"{'drop_off': {'latitude': 55.840302, 'longitud...",/v3/places/5bc4936d898bdc002bfa5d46,"{'address': '45 Auckland Wynd', 'admin_region'...",Certified Professional Never Binge Again Maste...,{},Europe/London


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [24]:
# Copy the desired columns into the final dataframe
selected_poi_data_df = poi_data_df[['name','categories','location']].copy()

# Display results
selected_poi_data_df

,name,categories,location
0,Wilson Street Pantry,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...","{'address': '6 Wilson St', 'admin_region': 'Sc..."
1,The Dhabba,"[{'id': 13199, 'name': 'Indian Restaurant', 'i...","{'address': '44 Candleriggs', 'admin_region': ..."
2,Merchant Square,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...","{'address': '71-73 Albion St', 'admin_region':..."
3,City Halls,"[{'id': 10037, 'name': 'Concert Hall', 'icon':...","{'admin_region': 'Scotland', 'country': 'GB', ..."
4,Spitfire Espresso,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...","{'address': '127 Candleriggs', 'admin_region':..."
...,...,...,...
1051,London Cab Sales,"[{'id': 17010, 'name': 'Used Car Dealership', ...","{'address': '136 Boden St', 'admin_region': 'S..."
1052,Baltic Street Adventure Playground,"[{'id': 16037, 'name': 'Playground', 'icon': {...","{'address': '421 Baltic St', 'admin_region': '..."
1053,London Road Police Station,"[{'id': 12072, 'name': 'Police Station', 'icon...","{'address': '851 London Rd', 'admin_region': '..."
1054,Certified Professional Never Binge Again Maste...,"[{'id': 15003, 'name': 'Alternative Medicine C...","{'address': '45 Auckland Wynd', 'admin_region'..."


Extract the important values from the Category & Location dictionaries

In [25]:
# Function used in conjunction with .apply() to extract category "names" into a easier to read list
def get_category_names(row):
    # Load the dictionary from the categories column
    categories_dic = row['categories']
    
    # Extract the names of each category into a tuple
    # NOTE: Tried list before, but .drop_duplicates() does not work on that data type so I had to settle with tuples
    category_tuple = ()
    for category in categories_dic:
        category_tuple += (category['name'],) # Append the name to the category tuple
        
    # Return the tuple of category name
    return category_tuple

# Function used in conjunction with .apply() to extract the address from a location, as we do not require the other data
def get_address(row):
    # Load the dictionary from the location row
    location_dic = row['location']
    
    # Extract the address from the location dictionary
    poi_address = location_dic.get('address', '')
    
    return poi_address

# Use pandas .apply() dataframe function to extract the important values into new columns for much easier analysis
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html
selected_poi_data_df['category_names'] = selected_poi_data_df.apply(get_category_names, axis=1)
selected_poi_data_df['address'] = selected_poi_data_df.apply(get_address, axis=1)

# Display the resulting DataFrame
selected_poi_data_df

,name,categories,location,category_names,address
0,Wilson Street Pantry,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...","{'address': '6 Wilson St', 'admin_region': 'Sc...","(Café, Restaurant)",6 Wilson St
1,The Dhabba,"[{'id': 13199, 'name': 'Indian Restaurant', 'i...","{'address': '44 Candleriggs', 'admin_region': ...","(Indian Restaurant,)",44 Candleriggs
2,Merchant Square,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...","{'address': '71-73 Albion St', 'admin_region':...","(Bar, Restaurant)",71-73 Albion St
3,City Halls,"[{'id': 10037, 'name': 'Concert Hall', 'icon':...","{'admin_region': 'Scotland', 'country': 'GB', ...","(Concert Hall, Theater)",
4,Spitfire Espresso,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...","{'address': '127 Candleriggs', 'admin_region':...","(Coffee Shop, Restaurant)",127 Candleriggs
...,...,...,...,...,...
1051,London Cab Sales,"[{'id': 17010, 'name': 'Used Car Dealership', ...","{'address': '136 Boden St', 'admin_region': 'S...","(Used Car Dealership,)",136 Boden St
1052,Baltic Street Adventure Playground,"[{'id': 16037, 'name': 'Playground', 'icon': {...","{'address': '421 Baltic St', 'admin_region': '...","(Playground,)",421 Baltic St
1053,London Road Police Station,"[{'id': 12072, 'name': 'Police Station', 'icon...","{'address': '851 London Rd', 'admin_region': '...","(Police Station,)",851 London Rd
1054,Certified Professional Never Binge Again Maste...,"[{'id': 15003, 'name': 'Alternative Medicine C...","{'address': '45 Auckland Wynd', 'admin_region'...","(Alternative Medicine Clinic,)",45 Auckland Wynd


Drop the columns that are no longer needed & remove duplicates

In [30]:
# Drop columns
#selected_poi_data_df = selected_poi_data_df.drop(['categories','location'], axis=1) # - Commented out as I've already run before and keep getting 'doesn't exist' errors

# Remove duplicates (Will happen in cases when two bike stations are within the search radius of eachother)
selected_poi_data_df = selected_poi_data_df.drop_duplicates()


# Final data frame to be exported to .CSV
selected_poi_data_df

,name,category_names,address
0,Wilson Street Pantry,"(Café, Restaurant)",6 Wilson St
1,The Dhabba,"(Indian Restaurant,)",44 Candleriggs
2,Merchant Square,"(Bar, Restaurant)",71-73 Albion St
3,City Halls,"(Concert Hall, Theater)",
4,Spitfire Espresso,"(Coffee Shop, Restaurant)",127 Candleriggs
...,...,...,...
1042,Lidl,"(Grocery Store,)",19 Moss Rd
1043,Motus Commercials Glasgow,"(Car Dealership,)",131 Bogmoor Rd Govan
1044,Glasgow Centre For Reproductive Medicine,"(Healthcare Clinic,)",21 Fifty Pitches Way
1045,J K Electrical,"(Electrician,)","17 Fulbar Rd, Cardonald"


In [31]:
# Export to .csv
selected_poi_data_df.to_csv('../data/Foursquare_POIs.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [42]:
# Pull API key from enviroment variables
yelp_api_key = os.environ["YELP_API_KEY"]

# Create API request url
yelp_request_url = "https://api.yelp.com/v3/businesses/search?latitude=55.8581672&longitude=-4.245483"

# Headers & authorization headers needed for Yelp API acording to docs
# https://docs.developer.yelp.com/docs/fusion-intro
yelp_headers = {"accept": "application/json"}
yelp_headers['Authorization'] = "Bearer " + os.environ["YELP_API_KEY"]

yelp_response = requests.get(yelp_request_url, headers=yelp_headers)

print(yelp_response)
print(yelp_response.json())

<Response [200]>
{'businesses': [{'id': 'UrnzTa6T3sLSVbbXLiYvQA', 'alias': 'paesano-pizza-glasgow', 'name': 'Paesano Pizza', 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/7KJazO4LNzm3gYq6hY3jQw/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/paesano-pizza-glasgow?adjust_creative=T1rHjlihHFyB-s1MAD5InQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=T1rHjlihHFyB-s1MAD5InQ', 'review_count': 154, 'categories': [{'alias': 'pizza', 'title': 'Pizza'}, {'alias': 'italian', 'title': 'Italian'}], 'rating': 4.5, 'coordinates': {'latitude': 55.8596504377382, 'longitude': -4.25060488821112}, 'transactions': [], 'price': '££', 'location': {'address1': '94 Miller Street', 'address2': '', 'address3': '', 'city': 'Glasgow', 'zip_code': 'G1 1DT', 'country': 'GB', 'state': 'GLG', 'display_address': ['94 Miller Street', 'Glasgow G1 1DT', 'United Kingdom']}, 'phone': '+441412585565', 'display_phone': '+44 141 258 5565', 'distance': 359.6795932482489}, {'id': 'JNp

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Bearer aPaoTKkz-vrobiMrjR0D65RtbdbgENolYnXowrqaOlcX7nXQbY67U8VYVBKh4CLfPGdxIS53ncuRrYOZvZZfM7edLI61pkGWirkEfU-_j7bp10nNEH_-W4ZxK6_zZHYx


Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating